In [ ]:
library(data.table) # version 1.14.6
library(dplyr)
# set working directory
setwd("/ch_progression/aric/epi/")

In [ ]:
# Load data
## 0/1 CH status
# aric_baseline_n_v05 <- fread("../pheno/aric_baseline_n_v05_N4189.pheno_ch_status_trajectory.23Mar2023.csv", header=T)
# aric_baseline_n_v05$dAge <- aric_baseline_n_v05$Age - aric_baseline_n_v05$age_base
#summary(aric_baseline_n_v05$dAge)
aric_baseline_n_v05_noPrevHeme <- fread("../pheno/aric_baseline_n_v05_N4187.pheno_ch_status.noHemeCA.9May2023.csv", header=T)
# 
aric_baseline_n_v05_noPrevHeme$dAge <- aric_baseline_n_v05_noPrevHeme$Age - aric_baseline_n_v05_noPrevHeme$age_base
summary(aric_baseline_n_v05_noPrevHeme$dAge)
nrow(aric_baseline_n_v05_noPrevHeme)
table(aric_baseline_n_v05_noPrevHeme$incident_CH)

## corrected lipid values
lipids_base <- fread("../pheno/aric_baseline_vanilla_02082023.csv", header=T, sep="\t")

## Update lipid values in mg/dl
aric_baseline_n_v05 <- merge(aric_baseline_n_v05_noPrevHeme[, c(1:63,68:112)], 
                                        lipids_base[, c(1,16:19)], 
                                        by.x="GWAS_ID", 
                                        by.y = "gwasid")

# names(aric_baseline_n_v05)
summary(aric_baseline_n_v05$ldl_base)
summary(aric_baseline_n_v05$chol_base)
summary(aric_baseline_n_v05$hdl_base)
summary(aric_baseline_n_v05$tg_base)


In [ ]:

# # Unadjusted: cont. variable
# chd_is_base== CHD or IS
aric_baseline_n_v05$chd_is_base <- ifelse(aric_baseline_n_v05$chd_base==1 | aric_baseline_n_v05$is_base==1, 1,
                                          ifelse(aric_baseline_n_v05$chd_base==0 | aric_baseline_n_v05$is_base==0,
                                                 0,NA))
table(aric_baseline_n_v05$chd_is_base, exclude = NULL)

 # ASCVD = c("chd", "is")
# aric_baseline_n_v05$ascvd_base <- ifelse(aric_baseline_n_v05$chd_base==1 | 
  #                                         aric_baseline_n_v05$is_base==1 , 1,
   #                                       ifelse(aric_baseline_n_v05$chd_base==0 | 
    #                                               aric_baseline_n_v05$is_base==0 |  
     #                                            is.na( aric_baseline_n_v05$is_base),0,NA))
# table(aric_baseline_n_v05$ascvd_base, exclude = NULL)
# aric_baseline_n_v05$ascvd_base[is.na(aric_baseline_n_v05$ascvd_base)] <- 0
# table(aric_baseline_n_v05$ascvd_base, exclude = NULL)
nrow(aric_baseline_n_v05)



In [ ]:
## corrected GPT version for missing data
# inverse_rank_normalize <- function(x) {
#  n <- sum(!is.na(x))
#  ranks <- rank(x, na.last = "keep")
#  normalized_values <- (ranks - 0.5) / n 
#  inverse_normalized_values <- qnorm(normalized_values)
#  return(inverse_normalized_values)
# }

### 
  ### source:  https://www.biostars.org/p/80597/ and the supplement of Yang et al. Nature 2012.
INT_yang2012 <- function(x){
  y<-qnorm((rank(x,na.last='keep')-0.5)/sum(!is.na(x)))
  return(y)
}

In [ ]:
## Scale
    # INT
aric_baseline_n_v05$chol_base_INT <- INT_yang2012(aric_baseline_n_v05$chol_base)

aric_baseline_n_v05$ldl_base_INT <- INT_yang2012(aric_baseline_n_v05$ldl_base)

aric_baseline_n_v05$hdl_base_INT <- INT_yang2012(aric_baseline_n_v05$hdl_base)

aric_baseline_n_v05$tg_base_INT <- INT_yang2012(aric_baseline_n_v05$tg_base)

aric_baseline_n_v05$nonHDL_base <- (aric_baseline_n_v05$chol_base - aric_baseline_n_v05$hdl_base)

aric_baseline_n_v05$nonHDL_base_INT <- INT_yang2012(aric_baseline_n_v05$nonHDL_base)

aric_baseline_n_v05$bmi_base_INT <- INT_yang2012(aric_baseline_n_v05$bmi_base)

    # TG/HDL-C
aric_baseline_n_v05$tg_to_hdl_base <- (aric_baseline_n_v05$tg_base/aric_baseline_n_v05$hdl_base)

aric_baseline_n_v05$tg_to_hdl_base_INT <- INT_yang2012(aric_baseline_n_v05$tg_to_hdl_base)



In [ ]:
# High LDL 
aric_baseline_n_v05$ldl_base_nomal_vs_high <- ifelse(aric_baseline_n_v05$ldl_base<160, 0, 
                                                     ifelse(aric_baseline_n_v05$ldl_base>=160, 1, NA) )
table(aric_baseline_n_v05$ldl_base_nomal_vs_high, exclude= NULL)

# Normal (<160 mg/dl) vs. High LDL-C (>=160 mg/dl) 
table( aric_baseline_n_v05$ldl_base_nomal_vs_high) 
# aric_baseline_n_v05$ldl_base_nomal_vs_high <- factor(aric_baseline_n_v05$ldl_base_nomal_vs_high, 
  #                                                   levels = c("<160", ">=160"))
str( aric_baseline_n_v05$ldl_base_nomal_vs_high) 

# Low HDL: 
aric_baseline_n_v05$hdl_base_low <- ifelse( (aric_baseline_n_v05$hdl_base>=40 & aric_baseline_n_v05$Gender=="M") | 
                                           (aric_baseline_n_v05$hdl_base>=50 & aric_baseline_n_v05$Gender=="F"), 0, 
                                                     ifelse( (aric_baseline_n_v05$hdl_base<40 & aric_baseline_n_v05$Gender=="M") | 
                                           (aric_baseline_n_v05$hdl_base<50 & aric_baseline_n_v05$Gender=="F"), 1, NA) )
table(aric_baseline_n_v05$hdl_base_low, exclude= NULL)

In [ ]:
# Dyslipidemia: 
# LDL-C>=160
# total Chol>=240
# Triglyceride >=200
# HDL-C<40 in Men and <50 in Women
# or use of Statin
table( (aric_baseline_n_v05$ldl_base>=160 & aric_baseline_n_v05$chol_base>=240 & aric_baseline_n_v05$tg_base>=200) & ( (aric_baseline_n_v05$Gender=="M" & aric_baseline_n_v05$hdl_base<40) | (aric_baseline_n_v05$Gender=="F" & aric_baseline_n_v05$hdl_base<50) ) | aric_baseline_n_v05$statin_base==1)

aric_baseline_n_v05$Dyslipidemia <- ifelse((aric_baseline_n_v05$ldl_base>=160 & 
                                            aric_baseline_n_v05$chol_base>=240 & 
                                            aric_baseline_n_v05$tg_base>=200) & 
                                           ( (aric_baseline_n_v05$Gender=="M" & 
                                              aric_baseline_n_v05$hdl_base<40) | 
                                            (aric_baseline_n_v05$Gender=="F" & 
                                             aric_baseline_n_v05$hdl_base<50) ) | 
                                           aric_baseline_n_v05$statin_base==1, 1, 0)

table(aric_baseline_n_v05$Dyslipidemia, exclude=NULL)

In [ ]:
summary(aric_baseline_n_v05[,c(114:125)])

In [ ]:
## 
# fwrite(aric_baseline_n_v05, "../pheno/aric_baseline_n_v05_N4187.pheno_ch_status.noHemeCA.correct_lipids.Jun3May2023.csv", 
 #  row.names = F, col.names = T, sep=",")

In [ ]:
aric_baseline_n_v05 <- fread("../pheno/aric_baseline_n_v05_N4187.pheno_ch_status.noHemeCA.correct_lipids.Jun3May2023.csv", header=T)

nrow(aric_baseline_n_v05)

ncol(aric_baseline_n_v05)
ls()
names(aric_baseline_n_v05)

In [ ]:
summary(aric_baseline_n_v05[,c(114:125)])

In [ ]:
### Exclude Prev. CH 
aric_baseline_n_v05 <- subset(aric_baseline_n_v05, !is.na(aric_baseline_n_v05$incident_CH))
nrow(aric_baseline_n_v05)

In [ ]:
## Scale
### 
  ### source:  https://www.biostars.org/p/80597/ and the supplement of Yang et al. Nature 2012.
INT_yang2012 <- function(x){
  y<-qnorm((rank(x,na.last='keep')-0.5)/sum(!is.na(x)))
  return(y)
}
    # INT
aric_baseline_n_v05$chol_base_INT <- INT_yang2012(aric_baseline_n_v05$chol_base)

aric_baseline_n_v05$ldl_base_INT <- INT_yang2012(aric_baseline_n_v05$ldl_base)

aric_baseline_n_v05$hdl_base_INT <- INT_yang2012(aric_baseline_n_v05$hdl_base)

aric_baseline_n_v05$tg_base_INT <- INT_yang2012(aric_baseline_n_v05$tg_base)

aric_baseline_n_v05$nonHDL_base <- (aric_baseline_n_v05$chol_base - aric_baseline_n_v05$hdl_base)

aric_baseline_n_v05$nonHDL_base_INT <- INT_yang2012(aric_baseline_n_v05$nonHDL_base)

aric_baseline_n_v05$bmi_base_INT <- INT_yang2012(aric_baseline_n_v05$bmi_base)

    # TG/HDL-C
aric_baseline_n_v05$tg_to_hdl_base <- (aric_baseline_n_v05$tg_base/aric_baseline_n_v05$hdl_base)

aric_baseline_n_v05$tg_to_hdl_base_INT <- INT_yang2012(aric_baseline_n_v05$tg_to_hdl_base)


In [ ]:
## Save dataframe used in the final glm analysis
# fwrite(aric_baseline_n_v05, "../pheno/aric_baseline_n_v05_N3730.pheno_ch_status.noHemeCA.correct_lipids.FinalDataset_4_glm.July132023.csv", 
  # row.names = F, col.names = T, sep=",")


In [ ]:
names(aric_baseline_n_v05)

# Supplementary Figure 5 | 

## Univariable association (logistic regression) between baseline risk factors and incident CHIP.

## Un-adjusted model: GLM

In [ ]:
# cat(gsub(pattern = ", ", replacement = ",", x = toString(
 # c("Dataset","Outcome", "Exposure","Beta", "SE", "t-stat", "P"))),
 # file = "final_glm.univariable.incident_ch.2023Jul07.csv", append = F, fill = T)

cat(gsub(pattern = ", ", replacement = ",", x = toString(
  c("Dataset","Outcome", "Exposure","Beta", "SE", "z-value", "P", "Cases", "Controls", "N"))),
  file = "final_glm.univariable.incident_ch.2024Jul22.csv", append = F, fill = T)

In [ ]:
exposures <- c("age_base",  "bmi_base_INT",   
               "chol_base_INT", "ldl_base_INT",
               "hdl_base_INT", "tg_base_INT",
               "nonHDL_base_INT", "tg_to_hdl_base_INT",
               "ldl_base_nomal_vs_high", "Dyslipidemia",
               "hdl_base_low",
               "Sex", "race_BW", "ever_smoke", 
               "dm_126_base", "htn_5_base", 
               "chd_is_base")

ch_phenotype <- c("incident_CH", 
                  "incident_DNMT3A",
                  "incident_TET2",
                  "incident_ASXL1",
                  "incident_SF",
                  "incident_DDR")

##
for(i in exposures){
  
  for (j in ch_phenotype){
    cat("outcome:",j," exposure:", i,"\n")
    # remove NA
    model1 <- summary(aric_baseline_n_v05 %>% filter(!is.na(get(i)) & !is.na(get(j))) %>%
                        glm(get(j) ~  get(i), 
                            data = ., family = "binomial"))
            
#    cat( gsub(pattern = ", ", replacement = ",", x = toString(
 #     c("Univariable", paste0(j), paste0(i), 
  #      model1$coefficients[2,1:4]) ) ), 
  #    file = "final_glm.univariable.incident_ch.2023Jul07.csv", append = T, fill = T)
      
######## addeded for case-control number 
      # Extract the data used in the model
model_data <- model.frame(model1)

# Count the number of cases and controls
case_control_count <- table(model_data[[1]])

# Print number of cases and controls
cat(paste0(j),"~", paste0(i), ": Number of controls:", case_control_count[1], "\n")

cat(paste0(j),"~", paste0(i), ": Number of cases:", case_control_count[2], "\n")

      
      
    cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("Univariable", paste0(j), paste0(i), 
        model1$coefficients[2,1:4], 
        case_control_count[2], 
        case_control_count[1], 
        length(model_data[[1]])) ) ), 
      file = "final_glm.univariable.incident_ch.2024Jul22.csv", append = T, fill = T)
    
  }
}

In [ ]:
sd(aric_baseline_n_v05$ldl_base_INT, na.rm = T)
sd(aric_baseline_n_v05$tg_base_INT, na.rm = T)
sd(aric_baseline_n_v05$hdl_base_INT, na.rm = T)
sd(aric_baseline_n_v05$chol_base_INT, na.rm = T)
sd(aric_baseline_n_v05$tg_to_hdl_base_INT, na.rm = T)
plot(density(aric_baseline_n_v05$nonHDL_base_INT, na.rm = T), main="non-HDL-C")
plot(density(aric_baseline_n_v05$hdl_base_INT, na.rm = T), main="HDL-C")
plot(density(aric_baseline_n_v05$ldl_base_INT, na.rm = T), main="LDL-C")
plot(density(aric_baseline_n_v05$tg_base_INT, na.rm = T), main="TG-C")
plot(density(aric_baseline_n_v05$tg_to_hdl_base_INT, na.rm = T), main="TG-to-HDL-C")

# Figure 4 

# Supplymentary Fig 6 | 

## Association of clinical cardiovascular risk factors with incident clonal hematopoiesis of indeterminate potential (CHIP).


## Adjusted model:
### all exposures:
#### adjusted for age, Sex, Race, Smoking, bmi, ldl-c, hdl-c, t2d, htn, ascvd, chol_med, batch(visit,center)

In [ ]:
# cat(gsub(pattern = ", ", replacement = ",", x = toString(
 # c("Dataset","Outcome", "Exposure","Beta", "SE", "t-stat", "P"))),
 # file = "final_glm.multivariable.incident_ch.2023Jul07.csv", append = F, fill = T)

cat(gsub(pattern = ", ", replacement = ",", x = toString(
  c("Dataset","Outcome", "Exposure","Beta", "SE", "z-value", "P", "Cases", "Controls", "N"))),
  file = "final_glm.multivariable.incident_ch.2024Jul22.csv", append = F, fill = T)

In [ ]:
# Outcomes
ch_phenotype <- c("incident_CH", 
                  "incident_DNMT3A",
                  "incident_TET2",
                  "incident_ASXL1",
                  "incident_SF",
                  "incident_DDR")

# Exposures
test_exposures <- c("age_base", "Sex", "race_BW", 
                    "ever_smoke", "bmi_base_INT", 
                    "nonHDL_base_INT", "hdl_base_INT", 
                    "dm_126_base", "htn_5_base", 
                    "chd_is_base")

for (j in ch_phenotype){
  for (k in 1:length(test_exposures)) {
      
    cat("outcome:",j," exposure:", test_exposures[k],"\n")
      
    model3 <- summary(aric_baseline_n_v05 %>% 
                        filter(!is.na(get(j))) %>% 
                        glm(get(j) ~ 
                            age_base + Sex + race_BW + 
                            ever_smoke + bmi_base_INT + 
                            nonHDL_base_INT + hdl_base_INT + 
                            dm_126_base + htn_5_base + chd_is_base +  
                            chol_med_base + Center + v2_vs_other, 
                            data = ., family="binomial"))
      
#    cat( gsub(pattern = ", ", replacement = ",", x = toString(
 #     c("Adjusted", paste0(j), paste0(test_exposures[k]),
 #       model3$coefficients[k+1,1:4]) ) ),
 #     file = "final_glm.multivariable.incident_ch.2023Jul07.csv", 
 #     append = T, fill = T)
      
      
      
      
      ######## addeded for case-control number 
      # Extract the data used in the model
model_data <- model.frame(model3)

# Count the number of cases and controls
      table(model_data[[1]], exclude=NULL)
      
case_control_count <- table(model_data[[1]])

# Print number of cases and controls
cat("total N=", length(model_data[[1]]))
      
cat(paste0(j),"~", paste0(test_exposures[k]), ": Number of controls:", case_control_count[1], "\n")

cat(paste0(j),"~", paste0(test_exposures[k]), ": Number of cases:", case_control_count[2], "\n")

     cat( gsub(pattern = ", ", replacement = ",", x = toString(
      c("Adjusted", paste0(j), paste0(test_exposures[k]),
       model3$coefficients[k+1,1:4], 
        case_control_count[2], 
        case_control_count[1], 
        length(model_data[[1]])) ) ),
     file = "final_glm.multivariable.incident_ch.2024Jul22.csv", 
     append = T, fill = T)  
      
          
  }
}
